### Import Packages


In [1]:
%pip install mediapipe opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import mediapipe as mp
import cv2
import numpy as np


### Calculate the angle between joints 

In [3]:
# Calculate the angle between three points using the cosine rule
def calculate_angle(a, b, c):


    a = np.array([a.x, a.y])
    b = np.array([b.x, b.y])
    c = np.array([c.x, c.y])

    radians = np.arccos(np.dot(b - a, c - b) / (np.linalg.norm(b - a) * np.linalg.norm(c - b)))
    angle = np.degrees(radians)

    return angle

In [4]:
# Count the number of bicep curls performed in real-time using Mediapipe
def count_bicep_curls():
    
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils
    cap = cv2.VideoCapture(0)
    
    # cards constraints
    card_width, card_height = 150, 80
    card_x, card_y = 10, 10
    card_color = (255, 153, 13)
   
    
    with mp_pose.Pose(model_complexity = 2 ,min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        curl_count = 0
        is_curling = False
        prev_angle = None
        prev_angle1 = None

        while True:
            ret, image = cap.read() #check wheather the frame is caught or not 

            image = cv2.flip(image, 1) # to correct the mirror view 

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 

            results = pose.process(image_rgb)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            if results.pose_landmarks is not None:
                left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
                left_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
                left_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
                right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
                right_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW]
                right_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]

                angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                angle1 = calculate_angle(right_shoulder, right_elbow, right_wrist)

                if prev_angle and prev_angle1 is not None:
                    delta = abs(angle - prev_angle)
                    # cv2.putText(image, f"Right Angle: {delta:.2f} degrees", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                    delta1 = abs(angle1 - prev_angle1)
                    # cv2.putText(image, f"left Angle: {delta1:.2f} degrees", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                    if not is_curling and delta > 20 and delta1 > 20:
                        is_curling = True
                    elif is_curling and delta < 10 and delta1 < 10:
                        is_curling = False

                prev_angle = angle
                prev_angle1 = angle1

                if is_curling and angle > 160 and angle1 > 160:
                    curl_count += 1
                    is_curling = False
                
            cv2.rectangle(image, (card_x, card_y), (card_x + card_width, card_y + card_height), card_color, -1)

            # Display the text inside the card
            text_x, text_y = card_x + 10, card_y + 30
            text_line1 = 'Bicep Curl'
            text_line2 = f'Count: {curl_count}'
            line_spacing = 30
            font_scale = 0.7
            font_color = (255, 255, 255)

            cv2.putText(image, text_line1, (text_x, text_y), cv2.FONT_HERSHEY_COMPLEX, font_scale, font_color, 2, cv2.LINE_AA)
            cv2.putText(image, text_line2, (text_x, text_y + line_spacing), cv2.FONT_HERSHEY_COMPLEX, font_scale, font_color, 2, cv2.LINE_AA)
            

            cv2.imshow('Bicep Curl counter', image)
          
            
            if(cv2.waitKey(1) == ord('q')):
                break

    cap.release()
    cv2.destroyAllWindows()

   

    return curl_count

In [5]:
count =count_bicep_curls()
print(count)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)>